In [0]:
import IndividualClass as GN
from numpy.random import choice
import operator
import random
import public
import time
from joblib import Parallel, delayed
import copy_reg
import types
import multiprocessing

def calFitness(item):
    return item.Fitness()

def InitialPopulation(number):
    population = list()
    for i in range(number):
        individual = GN.Individual()
        population.append(individual)
    return population

def FitnessSorted(population):

    print "parents:"
    s = time.time()
    result = Parallel(n_jobs = 8)(delayed(calFitness)(individual) for individual in population)
    print "100 generation: ",time.time()-s
    i = 0
    fitness = list()
    for item in population:
        print i+1,") ",item.feature
        numOne = result[i][2]
        fitness.append(result[i][0]+(numOne/50))
        print "MAD: ",result[i][0],", MSE: ",result[i][1][0] , ", RMSE: ",result[i][1][1]  , ", Rsquared: ",result[i][1][2] 
        print
        i+=1  
        
    dict_indiFit = dict(zip(population[:],fitness[:]))
    return sorted(dict_indiFit.items(), key=operator.itemgetter(1))

def FitnessSortedChild(population):
    print "child:"
    result = Parallel(n_jobs = 8)(delayed(calFitness)(individual) for individual in population)
    i = 0
    fitness = list()
    for item in population:
        print i+1,") ",item.feature
        numOne = result[i][2]
        fitness.append(result[i][0]+(numOne/50))
        print "MAD: ",result[i][0],", MSE: ",result[i][1][0] , ", RMSE: ",result[i][1][1]  , ", Rsquared: ",result[i][1][2] 
        print
        i+=1  
        
    dict_indiFit = dict(zip(population[:],fitness[:]))
    return sorted(dict_indiFit.items(), key=operator.itemgetter(1))
  

def CalProbability(dic_individualFitness):
    probability , prob = list() , list()
    total = 0
    key = list()
    for item in dic_individualFitness:
        key.append(item[0])
        total = total + (1/item[1])
        prob.append(1/item[1])

    for fit in prob:
        probability.append(fit/total)
    dict_indiProb = dict(zip(key,probability))
    return dict_indiProb


def Selection(popRanked, eliteSize):
    selectionResults = []
    probs = CalProbability(popRanked)
    bestParent = []
    for i in range(0, eliteSize):
        selectionResults.append(popRanked[i][0])
        bestParent.append(popRanked[i])
    remain = choice(probs.keys(),len(popRanked) - eliteSize , p = probs.values())
    
    for i in range(len(popRanked) - eliteSize):
        selectionResults += [remain[i]]
    return selectionResults,bestParent

def getKey(item):
    return item[1]
  
def BreedPopulation(matingpool, bestparent, eliteSize):
    childrens = []
    length = len(matingpool)
    pool = random.sample(matingpool, len(matingpool))

    for i in range(0, eliteSize):
        childrens.append(bestparent[i])
    tempChild = list()
    for i in range(0, length,2):
        child1 , child2 = pool[i].Crossover(pool[i+1])
        tempChild.append(child1)
        tempChild.append(child2)
    mutateChild = MutatePopulation(tempChild)
    childRank = FitnessSortedChild(mutateChild)
    temp = childRank[:length-eliteSize]
    for item in temp:
        childrens.append(item)
      
    nextGeneration = sorted(childrens, key=getKey)
    return nextGeneration


def MutatePopulation(population):
    mutatedPop = []

    for indi in population:
        mutatedInd = indi.Mutation()
        mutatedPop.append(mutatedInd)
    return mutatedPop


def NextGeneration(currentGen, eliteSize,k):
    popRanked = currentGen
    matingpool , bestParent = Selection(popRanked, eliteSize)
    print k," generation"
    popRankedNext = BreedPopulation(matingpool, bestParent,eliteSize)
    return popRanked,popRankedNext
  

def FirstGeneration(currentGen, eliteSize):
    popRanked = FitnessSorted(currentGen)
    matingpool , bestParent = Selection(popRanked, eliteSize)
    print 2,' childeren generation'
    popRankedNext = BreedPopulation(matingpool, bestParent,eliteSize)
    return popRanked,popRankedNext


def geneticAlgorithmPlot(popSize, eliteSize, generations):
    
    pop = InitialPopulation(popSize)
    progressBest = []
    progressWorst = []
    progressMean = []
    print 1," generation"
    #mutationR = 10
    start_time = time.time()
    print "start time: ",start_time
    Current , Next = FirstGeneration(pop, eliteSize)
    n = 1
    print "****************************************************************"
    for indi in Next:
        print n,"  ) ",indi[1], "  ---> ",indi[0].feature
        n+=1
    progressBest.append(Current[0][1])
    progressWorst.append(Current[-1][1])
    sum = 0
    for i in range(0, popSize):
        sum += Current[i][1]
    mean = sum / popSize
    progressMean.append(mean)
    
    for i in range(2, generations+1):
        Current , Next = NextGeneration(Next, eliteSize,i)
        n = 1
        print "****************************************************************"
        for indi in Next:
            print n,"  ) ",indi[1], "  ---> ",indi[0].feature
            n+=1
    
        progressBest.append(Current[0][1])
        progressWorst.append(Current[-1][1])
        sum = 0
        for i in range(0, popSize):
            sum += Current[i][1]
        mean = sum / popSize
        progressMean.append(mean)
    print "final time: ",time.time()-start_time
    progressBest.append(Next[0][1])
    progressWorst.append(Next[-1][1])
    sum = 0
    for i in range(0, popSize):
        sum += Next[i][1]
    mean = sum / popSize
    progressMean.append(mean)
    print "progressBest = ",progressBest
    print "progressWorst = ",progressWorst
    print "progressMean = ",progressMean
    bestObj = Next[0][0]
    bestFeature = bestObj.feature
    bestMAD = Next[0][1]
    bestRes = bestObj.scores
    return bestFeature,bestMAD,bestRes

    
bestInd , bestMAD , otherRes= geneticAlgorithmPlot(popSize= 100, eliteSize=10, generations=100)
print "best chromosom in final generation with len: " ,len(bestInd)
print bestInd
print "Best MAD on Train: ", bestMAD
print "MSE , RMSE , R2:  ",otherRes
Xt , Yt = public.LoadPartOfDataTest(f = bestInd,c = 2)
print "On Test:"
result = public.EvaluationTest(Xt,Yt,f=bestInd,c=2)
print "MAD: ",result[0],", MSE: ",result[1] , ", RMSE: ",result[2] , ", Rsquared: ",result[3]
